## **Начнем с загрузки датасетов**

Структура наших датасетов:

1. **Quality of life** - наш основной датасет.

   Изначально в нем представлено 136 стран, каждой из которых присвоен рейтинг по 100-бальной шкале по следующим показателям:
   
   1) **Стабильность** - политическая и экономическая стабильность
   2) **Права** - правовая система государства, ее прозрачность и справедливость, права человека, свобода слова
   3) **Здоровье** - качество системы здравоохранения
   4) **Безопасность** - отражает уровень преступности, качество защиты граждан силовыми структурами от преступлений
   5) **Климат** - под ним стоит подразумевать экологическую обстановку в стране
   6) **Стоимость жизни**
   7) **Привлекательность страны для эмиграции**

   Датасет взят со следующего ресурса: https://www.worlddata.info/quality-of-life.php(переделать на гиперссылку). Там можно подробнее почитать про методику оценки каждого из показателей.

   На взгляд нашей команды, каждый из этих показателей по своему важен и влияет на продолжительность жизни, поэтому мы решили исключить умножение каждого из параметров на определенный коэффициент для подсчета итоговой оценки и присваивание ранга стране по нему, так как это не совсем объективно.

2. **Life expectancy** - в нем представлена средняя продолжительность жизни по странам и годам. Мы из него возьмем последний доступный год для каждой страны(в основном это будет 2021, самый ранний вариант - 2019).
3. **Inflation** - в нем представлена инфляция по странам и годам. Возьмем инфляцию за 2021 год.
4. **GDP** - в нем представлен ВВП по странам и годам. Возьмем ВВП за 2021 год.
5. **Gini** - индекс Джини по странам и годам. Он лежит в пределах от 0 до 100 и отражает уровень неравенства в стране(выше показатель - выше неравенство)
6. **Education** - процент людей по странам, имеющих хотя бы базовое образование.

Теперь, когда мы определились с датасетами и их наполнением - можно их загружать и приводить в красивый вид.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import scipy.stats as stats
from scipy.stats import pearsonr, spearmanr, linregress
from typing import Any, Dict, Tuple
from scipy.stats._stats_py import PearsonRResult, SignificanceResult
from scipy.stats._stats_mstats_common import LinregressResult
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
life_exp: pd.DataFrame = pd.read_csv('/content/life-expectancy.csv')
quality_life: pd.DataFrame = pd.read_csv('/content/quality_life.csv')
inflation: pd.DataFrame = pd.read_csv('/content/inflation (1).csv')
gdp: pd.DataFrame = pd.read_csv('/content/GDP (1).csv')
gini: pd.DataFrame = pd.read_csv('/content/gini.csv')
education: pd.DataFrame = pd.read_csv('/content/education.csv')

## **Соединение датасетов**

Решил объединить данные разделы, так как с этого момента буду рассказывать какие-то свои мысли о данных. Как и обещал, основным датасетом будет **quality_life**, но с некоторыми изменениями: убираем ранг страны, итоговую общую оценку и сортируем страны в алфавитном порядке. Летс гоу!

In [4]:
quality_life: pd.DataFrame
quality_life = quality_life.drop(columns=['Unnamed: 0', 'Rank', 'TotalQuality of life(100%)'], errors='ignore')

quality_life = quality_life.rename(columns={
    'Stability(15%)': 'Stability',
    'Rights(20%)': 'Rights',
    'Health(15%)': 'Health',
    'Safety(10%)': 'Safety',
    'Climate(15%)': 'Climate',
    'Costs(15%)': 'Costs',
    'Popularity(10%)': 'Popularity'
})

quality_life = quality_life.sort_values(by='Country').reset_index(drop=True)

quality_life

,Country,Stability,Rights,Health,Safety,Climate,Costs,Popularity
0,Afghanistan,28,9,20,19,58,63,63
1,Albania,58,43,64,84,55,57,26
2,Algeria,35,24,61,97,78,55,36
3,Angola,27,24,10,55,63,53,31
4,Argentina,41,44,79,78,77,41,38
...,...,...,...,...,...,...,...,...
132,United States,63,81,71,85,66,37,67
133,Uruguay,67,79,75,82,77,47,30
134,Vietnam,67,35,54,76,52,51,33
135,Zambia,34,32,18,60,68,58,47


Далее присоединим продолжительность жизни к основе, но до этого отфильтруем этот датасет. Выведем данные за последний доступный год(спойлер - это всегда будет 2021, кроме уже несуществующих стран, которые нас не интересуют), удалим ненужные колонки, переименуем столбцы.

In [5]:
life: pd.DataFrame
life = life_exp.loc[life_exp.groupby('Entity')['Year'].idxmax()]

life = life.drop(columns=['Code', 'Year'], errors='ignore')

life = life.rename(columns={
    'Entity': 'Country',
    'Period life expectancy at birth - Sex: all - Age: 0': 'Life expectancy'
})

life

,Country,Life expectancy
71,Afghanistan,61.98
143,Africa,61.66
217,Albania,76.46
289,Algeria,76.38
364,American Samoa,72.54
...,...,...
20459,Western Sahara,70.78
20531,World,71.05
20610,Yemen,63.75
20682,Zambia,61.22


Теперь соединим датасеты:

In [6]:
quality_life = quality_life.merge(life, how='left', on='Country')

С этого момента присоединение буду расписывать тезисно, чтобы не отнимать мое время(и проверяющего): в инфляции брал данные за 2021 год(если их не было - последние доступные по стране).

Потом просто присоединил к основному датасету.

In [7]:
inflation: pd.DataFrame
inflation = inflation[inflation['Year'] <= 2021]
inflation = inflation.loc[inflation.groupby('Country')['Year'].idxmax()]

inflation = inflation.drop(columns=['Country Code', 'Year'])
quality_life = quality_life.merge(inflation, how='left', on='Country')

quality_life

,Country,Stability,Rights,Health,Safety,Climate,Costs,Popularity,Life expectancy,Inflation
0,Afghanistan,28,9,20,19,58,63,63,61.98,2.30
1,Albania,58,43,64,84,55,57,26,76.46,2.04
2,Algeria,35,24,61,97,78,55,36,76.38,7.23
3,Angola,27,24,10,55,63,53,31,61.64,25.75
4,Argentina,41,44,79,78,77,41,38,75.39,NaN
...,...,...,...,...,...,...,...,...,...,...
132,United States,63,81,71,85,66,37,67,77.20,4.70
133,Uruguay,67,79,75,82,77,47,30,75.44,7.75
134,Vietnam,67,35,54,76,52,51,33,73.62,NaN
135,Zambia,34,32,18,60,68,58,47,61.22,22.02


По ВВП берем 2021 год.

In [8]:
gdp: pd.DataFrame
gdp = gdp[['Country', '2021']]
gdp = gdp.rename(columns={'2021': 'GDP'})

quality_life = quality_life.merge(gdp, how='left', on='Country')

quality_life

,Country,Stability,Rights,Health,Safety,Climate,Costs,Popularity,Life expectancy,Inflation,GDP
0,Afghanistan,28,9,20,19,58,63,63,61.98,2.30,14583135237.00
1,Albania,58,43,64,84,55,57,26,76.46,2.04,17930565119.00
2,Algeria,35,24,61,97,78,55,36,76.38,7.23,163472233246.00
3,Angola,27,24,10,55,63,53,31,61.64,25.75,65685435100.00
4,Argentina,41,44,79,78,77,41,38,75.39,NaN,487227125386.00
...,...,...,...,...,...,...,...,...,...,...,...
132,United States,63,81,71,85,66,37,67,77.20,4.70,23315080560000.00
133,Uruguay,67,79,75,82,77,47,30,75.44,7.75,61412268249.00
134,Vietnam,67,35,54,76,52,51,33,73.62,NaN,366137590718.00
135,Zambia,34,32,18,60,68,58,47,61.22,22.02,22147649569.00


По Джини берем последний доступный год. Этот самый год чаще всего лежит в 10-х годах, что является вполне допустимым для целей EDA, так как даже за 10 лет неравенство в стране не может сильно измениться.

In [9]:
gini: pd.DataFrame
gini = gini.loc[gini.groupby('Country')['Year'].idxmax()]
gini = gini.drop(columns=['Year'])
gini = gini.rename(columns={
    'Gini coefficient': 'Gini'
})

quality_life = quality_life.merge(gini, how='left', on='Country')

quality_life

,Country,Stability,Rights,Health,Safety,Climate,Costs,Popularity,Life expectancy,Inflation,GDP,Gini
0,Afghanistan,28,9,20,19,58,63,63,61.98,2.30,14583135237.00,NaN
1,Albania,58,43,64,84,55,57,26,76.46,2.04,17930565119.00,0.29
2,Algeria,35,24,61,97,78,55,36,76.38,7.23,163472233246.00,0.28
3,Angola,27,24,10,55,63,53,31,61.64,25.75,65685435100.00,0.51
4,Argentina,41,44,79,78,77,41,38,75.39,NaN,487227125386.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
132,United States,63,81,71,85,66,37,67,77.20,4.70,23315080560000.00,0.41
133,Uruguay,67,79,75,82,77,47,30,75.44,7.75,61412268249.00,0.41
134,Vietnam,67,35,54,76,52,51,33,73.62,NaN,366137590718.00,0.36
135,Zambia,34,32,18,60,68,58,47,61.22,22.02,22147649569.00,0.51


И напоследок, присоединим данные по образованию, берем данные по последнему году(2020 в основном).


In [10]:
education: pd.DataFrame
education = education.loc[education.groupby('Entity')['Year'].idxmax()]
education = education.drop(columns=['Code',
                                    'Share of population with no formal education, 1820-2020',
                                    'Year'])
education = education.rename(columns={
    'Share of population with some formal education, 1820-2020': 'Education level',
    'Entity': 'Country'
})

quality_life = quality_life.merge(education, how='left', on='Country')

quality_life

,Country,Stability,Rights,Health,Safety,Climate,Costs,Popularity,Life expectancy,Inflation,GDP,Gini,Education level
0,Afghanistan,28,9,20,19,58,63,63,61.98,2.30,14583135237.00,NaN,NaN
1,Albania,58,43,64,84,55,57,26,76.46,2.04,17930565119.00,0.29,99.00
2,Algeria,35,24,61,97,78,55,36,76.38,7.23,163472233246.00,0.28,87.00
3,Angola,27,24,10,55,63,53,31,61.64,25.75,65685435100.00,0.51,NaN
4,Argentina,41,44,79,78,77,41,38,75.39,NaN,487227125386.00,NaN,98.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,United States,63,81,71,85,66,37,67,77.20,4.70,23315080560000.00,0.41,100.00
133,Uruguay,67,79,75,82,77,47,30,75.44,7.75,61412268249.00,0.41,99.00
134,Vietnam,67,35,54,76,52,51,33,73.62,NaN,366137590718.00,0.36,95.00
135,Zambia,34,32,18,60,68,58,47,61.22,22.02,22147649569.00,0.51,95.00


## **Предобработка данных**

На мой взгляд, в нашем случае частичная предобработка была еще на этапе соединения датасетов, поэтому не стоит пренебрегать чтением предыдущего раздела)

Давайте посмотрим на основные характеристики итогового датасета.

In [11]:
quality_life.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          137 non-null    object 
 1   Stability        137 non-null    int64  
 2   Rights           137 non-null    int64  
 3   Health           137 non-null    int64  
 4   Safety           137 non-null    int64  
 5   Climate          137 non-null    int64  
 6   Costs            137 non-null    int64  
 7   Popularity       137 non-null    int64  
 8   Life expectancy  133 non-null    float64
 9   Inflation        120 non-null    float64
 10  GDP              123 non-null    float64
 11  Gini             120 non-null    float64
 12  Education level  126 non-null    float64
dtypes: float64(5), int64(7), object(1)
memory usage: 14.0+ KB


Из хорошего: все данные правильного типа, числовые - числового, буквенные - object. Стоит заметить, что все float округлены до двух знаков после запятой, чтобы не захламлять табличку, не сильно уменьшая точность расчетов.

Из плохого: довольно много пропусков, с которыми надо бороться. Давайте чуть пристальнее на них поглядим.

In [12]:
quality_life[quality_life['Life expectancy'].isna()]

,Country,Stability,Rights,Health,Safety,Climate,Costs,Popularity,Life expectancy,Inflation,GDP,Gini,Education level
18,Burma,31,24,32,63,50,73,33,NaN,NaN,NaN,NaN,NaN
27,Congo (Dem. Republic),32,7,6,10,54,60,36,NaN,NaN,NaN,NaN,NaN
59,Ivory Coast,47,36,11,58,52,57,37,NaN,NaN,NaN,NaN,NaN
123,Timor-Leste,65,37,42,42,72,67,23,NaN,3.70,3621222400.00,NaN,NaN


Что делать с такими пропусками? На наш взгляд, применять любое усреднение продолжительности жизни неэффективно - это показатель, который мы анализируем, поэтому он должен быть точным. Зато есть интернет, с помощью которого можно легко достать недостающие данные. Это немножко запарно, зато максимально эффективно:)

Также не поленюсь, и заполню другие пропуски тоже ручками, инфа важная, хоть и лень очень:(

In [13]:
# Burma
quality_life.at[18, 'GDP'] = 65000000000.00
quality_life.at[18, 'Life expectancy'] = 69.10
quality_life.at[18, 'Gini'] = 0.31
quality_life.at[18, 'Education level'] = 93.10
quality_life.at[18, 'Inflation'] = 21.40

# Congo
quality_life.at[27, 'GDP'] = 14830000000.00
quality_life.at[27, 'Life expectancy'] = 63.50
quality_life.at[27, 'Gini'] = 0.49
quality_life.at[27, 'Education level'] = 80.60
quality_life.at[27, 'Inflation'] = 19.00

# Ivory Coast
quality_life.at[59, 'GDP'] = 14830000000.00
quality_life.at[59, 'Life expectancy'] = 61.90
quality_life.at[59, 'Gini'] = 0.35
quality_life.at[59, 'Education level'] = 93.60
quality_life.at[59, 'Inflation'] = 17.40

# Timor-Leste
quality_life.at[123, 'GDP'] = 2080000000.00
quality_life.at[123, 'Life expectancy'] = 67.00
quality_life.at[123, 'Gini'] = 0.28
quality_life.at[123, 'Education level'] = 68.10
quality_life.at[123, 'Inflation'] = 6.10

# Egypt
quality_life.at[35, 'GDP'] = 404140000000.00
quality_life.at[35, 'Inflation'] = 13.89

# Gambia
quality_life.at[42, 'GDP'] = 2014000000.00
quality_life.at[42, 'Inflation'] = 7.37

# Hong Kong
quality_life.at[49, 'GDP'] = 368130000000.00
quality_life.at[49, 'Inflation'] = 1.57
quality_life.at[49, 'Gini'] = 0.35

# Iran
quality_life.at[54, 'GDP'] = 359100000000.00
quality_life.at[54, 'Inflation'] = 43.38

# Kyrgyztan
quality_life.at[66, 'GDP'] = 9350000000.00
quality_life.at[66, 'Inflation'] = 11.90

# Laos
quality_life.at[67, 'GDP'] = 18827000000.00
quality_life.at[67, 'Inflation'] = 3.75

# Macao
quality_life.at[73, 'GDP'] = 30500000000.00
quality_life.at[73, 'Inflation'] = 0.30
quality_life.at[73, 'Gini'] = 0.35

# Russia
quality_life.at[101, 'GDP'] = 1840000000000.00
quality_life.at[101, 'Inflation'] = 8.39

# Slovakia
quality_life.at[110, 'GDP'] = 115000000000.00
quality_life.at[110, 'Inflation'] = 3.15

# South Korea
quality_life.at[113, 'GDP'] = 2514520000000.00
quality_life.at[113, 'Inflation'] = 2.49

# South Korea
quality_life.at[127, 'GDP'] = 819865000000.00
quality_life.at[127, 'Inflation'] = 19.59

# Afghanistan
quality_life.at[0, 'Gini'] = 0.41
quality_life.at[0, 'Education level'] = 37.30

# Argentina
quality_life.at[4, 'Gini'] = 0.42
quality_life.at[4, 'Inflation'] = 50.94

# Bermuda
quality_life.at[13, 'Inflation'] = 3.00
quality_life.at[13, 'Education level'] = 77.70

# San Marino
quality_life.at[104, 'Education level'] = 100.00

# Oman
quality_life.at[92, 'Education level'] = 94.86

# Angola
quality_life.at[3, 'Education level'] = 72.40

# Sri Lanka
quality_life.at[115, 'Education level'] = 92.00

# Angola
quality_life.at[124, 'Education level'] = 67.00

# Vietnam
quality_life.at[134, 'Inflation'] = 1.84

По Бахрейну, Бермудах, Камбодже, Кувейту, Новой Зеландии, Оману, Сан-Марино, Саудовской Аравии, Сингапуру данных по индексу Джини нет, поэтому заполним данные поля средним значением, так как получить лучшее приближение не представляется возможным, а из-за большого объема выборки на итоговый результат это сильно не повлияет.

In [14]:
quality_life['Gini'] = quality_life['Gini'].fillna(quality_life['Gini'].mean())

Теперь приведем данные в вид, комфортный для анализа.

Во-первых, разделим на миллион ВВП, чтобы считать его в млн долларов(запись будет короче и комфортнее).

Во-вторых, умножим к/ф Джини на 100, чтобы считать его в более удобном виде в процентах.

In [15]:
quality_life['Gini'] = quality_life['Gini'] * 100
quality_life['GDP'] = quality_life['GDP'] / 1000000

Посмотрим на описание данных и со спокойной душой перейдем к следующему разделу:)

In [16]:
quality_life.describe()

,Stability,Rights,Health,Safety,Climate,Costs,Popularity,Life expectancy,Inflation,GDP,Gini,Education level
count,137.00,137.00,137.00,137.00,137.00,137.00,137.00,137.00,137.00,137.00,137.00,137.00
mean,57.58,50.69,58.12,76.78,53.23,52.34,37.98,73.04,10.41,696484.41,35.94,90.84
std,19.33,25.14,26.09,21.79,19.96,11.61,12.05,7.52,34.55,2577649.55,6.87,13.57
min,8.00,7.00,0.00,4.00,2.00,23.00,11.00,52.53,-0.77,843.85,24.11,37.30
25%,46.00,32.00,42.00,69.00,42.00,44.00,31.00,68.85,2.19,23131.94,31.50,88.00
50%,57.00,45.00,63.00,85.00,53.00,54.00,37.00,73.67,3.75,84061.40,35.00,97.00
75%,72.00,72.00,80.00,93.00,68.00,61.00,42.00,78.94,6.65,404140.00,39.18,99.00
max,93.00,100.00,100.00,100.00,95.00,76.00,73.00,85.47,359.09,23315080.56,63.03,100.00


## **Создание новых признаков**



In [17]:
print(quality_life[quality_life['Education level'] > 90].count())

Country            98
Stability          98
Rights             98
Health             98
Safety             98
Climate            98
Costs              98
Popularity         98
Life expectancy    98
Inflation          98
GDP                98
Gini               98
Education level    98
dtype: int64


Если посмотреть на наши данные, то можно заметить, что в большинстве стран мира показатель Education level больше 90. Это и логично, ведь он показывает процент населения, имеющих хотя бы базовое образование. Тогда мы можем создать отдельный признак, чтобы отделить страны с грамотным населением от стран с низким уровнем образования. Это будет более логично, так как при оценке страны смотрят на систему образования в целом.

In [18]:
quality_life['Literacy'] = (quality_life['Education level'] > 90).astype(int)
quality_life

,Country,Stability,Rights,Health,Safety,Climate,Costs,Popularity,Life expectancy,Inflation,GDP,Gini,Education level,Literacy
0,Afghanistan,28,9,20,19,58,63,63,61.98,2.30,14583.14,41.00,37.30,0
1,Albania,58,43,64,84,55,57,26,76.46,2.04,17930.57,29.42,99.00,1
2,Algeria,35,24,61,97,78,55,36,76.38,7.23,163472.23,27.62,87.00,0
3,Angola,27,24,10,55,63,53,31,61.64,25.75,65685.44,51.26,72.40,0
4,Argentina,41,44,79,78,77,41,38,75.39,50.94,487227.13,42.00,98.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,United States,63,81,71,85,66,37,67,77.20,4.70,23315080.56,41.33,100.00,1
133,Uruguay,67,79,75,82,77,47,30,75.44,7.75,61412.27,40.55,99.00,1
134,Vietnam,67,35,54,76,52,51,33,73.62,1.84,366137.59,36.09,95.00,1
135,Zambia,34,32,18,60,68,58,47,61.22,22.02,22147.65,51.48,95.00,1


Аналогичные рассуждения можно применить и к инфляции, что разница между тем в стране инфляция 5 или 8 процентных пунктов глобально не сильно влияет на состояние экономики и общественной жизни в стране. Поэтому исходя из макроэкономики поделим её на низкую (до 5%), умеренную (до 10%), галопирующую (до 100%) и гиперинфляцию (свыше 100% в год).

In [19]:
def classification(inflation) -> str:
    if inflation <= 5:
        return 'Creeping'
    elif 5 < inflation <= 10:
        return 'Walking'
    elif 10 < inflation <= 100:
        return 'Galloping'
    else:
        return 'Hyperinflation'
quality_life['Type of inflation'] = quality_life['Inflation'].apply(classification)

Рассуждая о ВВП, нетрудно заметить, что этот показатель не отражает богатство населения страны, так как весь доход от производства товаров и услуг может быть сосредоточен в руках узкого круга лиц. Тогда введем скорректированный ВВП с учётом неравенства и на единицу стоимости жизни, чтобы учесть уровень цен в разных странах.

$Corrected$ $GDP = \frac{GDP * (1 - Gini/100)}{costs + 1}$

Это позволит нам оценить "эффективный" ВВП, доступный для большинства населения.

In [20]:
quality_life['Corrected_GDP'] = (quality_life['GDP'] * (1 - quality_life['Gini'] / 100)) / (quality_life['Costs'] + 1)

## **Визуализация**

Начнём с самого простого. Давайте посмотрим на то, в каких странах самая высокая продолжительность жизни. Скорее всего там будет высокий уровень грамотности, низкая инфляция и развиты различные институты, а возможно и нет. Сейчас и узнаем :)

In [21]:
quality_life_sorted: pd.DataFrame = quality_life.sort_values(by='Life expectancy', ascending=False).head(15)

fig = go.Figure()

fig.add_trace(go.Bar(
    x = quality_life_sorted['Life expectancy'],
    y = quality_life_sorted['Country'],
    orientation = 'h',
    marker = dict(color='skyblue'),
    name = "Продолжительность жизни"
))

fig.update_layout(
    title = "Продолжительность жизни в разных странах мира",
    xaxis_title = "Продолжительность жизни",
    yaxis_title = "Страна",
    width = 900,
    height = 500,
    yaxis = dict(categoryorder = 'total ascending')
)


fig.show()

In [22]:
quality_life_sorted = quality_life.sort_values(by='Life expectancy', ascending=False).tail(15)
fig = go.Figure()

fig.add_trace(go.Bar(
    x = quality_life_sorted['Life expectancy'],
    y = quality_life_sorted['Country'],
    orientation = 'h',
    marker = dict(color='skyblue'),
    name = "Продолжительность жизни"
))

fig.update_layout(
    title = "Продолжительность жизни в разных странах мира",
    xaxis_title = "Продолжительность жизни",
    yaxis_title = "Страна",
    width = 900,
    height = 500,
    yaxis = dict(categoryorder='total ascending'),
)

fig.show()

Самая высокая продолжительность жизни на удивление оказалась в азиатских регионах (Гонконг, Макао, Япония, Южная Корея), Австралии и европейских госудрствах. Исходя из общеэкономических представлений, их объединяет высокий уровень благосостояния на душу населения, комфортный климат и высокий уровень медицины, образования и системы права.

Самая низкая продолжительность жизни, как и ожидалось, наблюдается в бедных африканских странах (практически все страны из топ-15 аутсайдеров расположены в Африке), хотя у них довольно хороший климат и низкая стоимость жизни, но видимо экономические факторы имеют куда больший вес))

In [23]:
df_gdp: pd.DataFrame = quality_life.sort_values(by='Corrected_GDP', ascending=False).head(15)

fig = go.Figure()

fig.add_trace(go.Pie(
    labels = df_gdp['Country'],
    values = df_gdp['Corrected_GDP'],
    textinfo = 'percent+label'
))

fig.update_layout(
    title = "Распределение скорректированного ВВП среди топ-15 стран",
    width = 900,
    height = 900
)
fig.show()

Давайте теперь посмотрим на наш новый признак - скорректированный ВВП с учетом неравенства и стоимости жизни. Если мы рассмотрим только топ-15 стран по данному показателю, то половина суммарного скорректированного ВВП приходится на Китай и США. Далее идут Япония, Франция и Германия, но уже с заметно сильным отрывом. Удивительно, что из всех этих стран только Япония была в лидерах по продолжительности жизни. Значит, всё-таки даже пусть и скорректированный ВВП не прям сильно связан с продолжительностью жизни.

In [24]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = quality_life['Health'],
    y = quality_life['Life expectancy'],
    mode = 'markers+text',
    marker = dict(size=9, color='blue', opacity=0.7),
    text = quality_life['Country'],
    textposition = 'top left',
    name = "Страны"
))

fig.update_layout(
    title = "Продолжительность жизни и качество системы здравоохранения",
    xaxis_title = "Health",
    yaxis_title = "Life expectancy",
    width = 1000,
    height = 600
)

fig.show()

Кажется это то, что мы искали. На диаграмме рассеивания явно видно, что между продолжительностью жизни и качеством системы здравоохранения есть положительная корелляция. Это и вполне логично, ведь для поддержания своей жизни в старости нужно хорошее качество медицинских услуг и их доступность. Что же касается явных лидеров: Гонконга и Макао, то это небольшие города-полугосударства  (Специальные административные районы), где изначально живут очень богатые люди, у которых есть деньги на качественные медицинские обследования и как следствие, там очень много частных хороших клиник с узкоспециализированными специалистами. Давайте теперь попробуем отыскать еще какие-то закономерности, посмотрев на разные графики.

In [25]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = quality_life['Climate'],
    y = quality_life['Popularity'],
    mode = 'markers',
    marker = dict(size=10, color='orange', opacity=0.8),
    hovertext = quality_life['Country'],
    hoverinfo = 'text+x+y',
    name = "Страны"
))

fig.update_layout(
    title = "Климат и популярность для эмиграции",
    xaxis_title = "Климат",
    yaxis_title = "Привлекательность страны",
    width = 1000,
    height = 600
)

fig.show()

In [26]:
fig = go.Figure()

for inflation_type in quality_life['Type of inflation'].unique():
    fig.add_trace(go.Box(
        y = quality_life[quality_life['Type of inflation'] == inflation_type]['Gini'],
        name = inflation_type,
        boxmean = True,
        marker = dict(color='blue', opacity=0.7)
    ))

fig.update_layout(
    title = "Экономическое неравенство в зависимости от типа инфляции",
    xaxis_title = "Тип инфляции",
    yaxis_title = "Gini",
    width = 900,
    height = 600
)

fig.show()

Исходя из ящика с усами, построенного для индекса Джини в зависимости от типа инфляции, медиана индекса Джини всегда находится на примерно одном и том же уровне вне зависимости от типа инфляции (33-36%). Самый большой интерквантильный размах у галлопирующей инфляции в то время, как у гиперинфляции он очень низкий и хвосты очень короткие.

In [27]:
quality_life['Institutions'] = (
    quality_life['Stability'] * quality_life['Rights'] * quality_life['Safety']
)

fig = go.Figure()

for inflation_type in quality_life['Type of inflation'].unique():
    fig.add_trace(go.Box(
        y = quality_life[quality_life['Type of inflation'] == inflation_type]['Institutions'],
        name = inflation_type,
        boxmean = True,
        marker = dict(color='blue', opacity=0.7)
    ))

fig.update_layout(
    title = "Связь развития институтов и типа инфляции",
    xaxis_title = "Тип инфляции",
    yaxis_title = "Institutions",
    width = 900,
    height = 600
)

fig.show()

Объединив политическую и экономическую стабильность, правовую систему государства и безопасность в один показатель, отражающий развитие институтов в конкретной стране, мы получили весьма интересные наблюдения. Гиперинфляция и галопирующая инфляция наблюдаются в странах с низким уровнем развития институтов, а наиболее благоприятные ползучая и умеренная инфляции соответствуют странам с более высоким институциональным развитием. Причем для перехода от умеренной инфляции (до 10%) к ползучей (до 5%) требуется сильно больший уровень стабильности, правовой системы и безопасности.

Хотя у нас и было предположение, что чем благоприятнее климат, тем более привлекательна страна в плане эмиграции, но всё же прямая такая зависимость явно отсутствует.

## **Гипотезы**

#### Гипотеза 1: Проверка гипотезы: Влияние уровня образования на продолжительность жизни

In [28]:
df_life = quality_life[['Education level', 'Life expectancy']].dropna()

shapiro_edu = stats.shapiro(df_life['Education level'])
shapiro_life = stats.shapiro(df_life['Life expectancy'])

pearson_corr: float
pearson_p: float
pearson_corr, pearson_p = stats.pearsonr(df_life['Education level'], df_life['Life expectancy'])
spearman_corr: float
spearman_p: float
spearman_corr, spearman_p = stats.spearmanr(df_life['Education level'], df_life['Life expectancy'])

In [29]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = df_life['Education level'],
    y = df_life['Life expectancy'],
    mode = 'markers',
    marker = dict(size=8, color='blue', opacity=0.7),
    hovertext = df_life.index,
    hoverinfo = 'text+x+y',
    name = "Страны"
))

fig.update_layout(
    title = "Связь уровня образования и продолжительности жизни",
    xaxis_title = "Уровень образования (%)",
    yaxis_title = "Продолжительность жизни (лет)",
    width = 900,
    height = 600
)

fig.show()

In [30]:
life_results = {
    "Shapiro-Wilk Education p-value": shapiro_edu.pvalue,
    "Shapiro-Wilk Life Expectancy p-value": shapiro_life.pvalue,
    "Pearson Corr": pearson_corr,
    "Pearson p-value": pearson_p,
    "Spearman Corr": spearman_corr,
    "Spearman p-value": spearman_p
}

life_results

{'Shapiro-Wilk Education p-value': 2.4589448832601933e-15,
 'Shapiro-Wilk Life Expectancy p-value': 0.0021854707734657868,
 'Pearson Corr': 0.5936330048635492,
 'Pearson p-value': 2.0893501788014177e-14,
 'Spearman Corr': 0.5839281986881277,
 'Spearman p-value': 6.913317214185513e-14}

Критерий Шапиро-Уилка показал, что данные не нормальны (p < 0.05).

Корреляция Пирсона (r = 0.59) и Спирмена (r = 0.58) указывает на умеренную положительную связь.

p-значения (< 0.0001) подтверждают статистическую значимость корреляции.

График показывает, что страны с высоким уровнем образования, как правило, имеют большую продолжительность жизни.

Вывод: Люди с лучшим образованием получают больше знаний о здоровье, имеют лучший доступ к медицинским услугам и более высокий уровень дохода, что увеличивает продолжительность жизни.

#### Гипотеза 2: Связь показателя здоровья с продолжительностью жизни

In [31]:
df_health: pd.DataFrame = quality_life[['Country', 'Health', 'Life expectancy']].dropna()

pearson_health: PearsonRResult = pearsonr(df_health['Health'], df_health['Life expectancy'])
spearman_health: SignificanceResult = spearmanr(df_health['Health'], df_health['Life expectancy'])

print(f"Pearson correlation (Health vs LifeExp): {pearson_health[0]:.4f}, p-value = {pearson_health[1]:.4f}")
print(f"Spearman correlation (Health vs LifeExp): {spearman_health.correlation:.4f}, p-value = {spearman_health.pvalue:.4f}")

reg_health: LinregressResult = linregress(df_health['Health'], df_health['Life expectancy'])

print(f"Linear regression: slope = {reg_health.slope:.4f}, intercept = {reg_health.intercept:.4f}")
print(f"R-squared = {reg_health.rvalue**2:.4f}, p-value (slope) = {reg_health.pvalue}")




Pearson correlation (Health vs LifeExp): 0.9174, p-value = 0.0000
Spearman correlation (Health vs LifeExp): 0.9007, p-value = 0.0000
Linear regression: slope = 0.2645, intercept = 57.6645
R-squared = 0.8417, p-value (slope) = 7.007944035680107e-56


In [32]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = df_health['Health'],
    y = df_health['Life expectancy'],
    mode = 'markers+text',
    hovertext = df_health['Country'],
    hoverinfo = 'text+x+y',
    marker = dict(size = 8, color = 'green', opacity = 0.7),
    name = "Страны"
))
fig.update_layout(
    title = "Здоровье VS продолжительность жизни",
    xaxis_title = "Качество здравоохранения",
    yaxis_title = "Ожидаемое кол-во жизни",
    width = 800,
    height = 600
)
fig.show()

Корреляция Пирсона (r = 0.917) и Спирмена (r = 0.901) говорит о сильной положительной связи.

Высокий R² = 0.842 означает, что 84.2% вариации продолжительности жизни объясняется качеством здравоохранения.

Наклон в линейной регрессии (0.264) показывает, что улучшение системы здравоохранения на 1 пункт повышает ожидаемую продолжительность жизни на 0.26 лет.

Вывод: Хорошая система здравоохранения способствует увеличению продолжительности жизни, снижению смертности и доступу к качественным медицинским услугам.

#### Гипотеза 3: Связь стоимости жизни и скорректированного ВВП

In [33]:
df_costs: pd.DataFrame = quality_life[['Country', 'Costs', 'Corrected_GDP']].dropna()

pearson_costs: PearsonRResult = pearsonr(df_costs['Costs'], df_costs['Corrected_GDP'])
spearman_costs: SignificanceResult = spearmanr(df_costs['Costs'], df_costs['Corrected_GDP'])

print(f"Pearson correlation (Costs vs Corrected GDP): {pearson_costs[0]:.4f}, p-value = {pearson_costs[1]:.4f}")
print(f"Spearman correlation (Costs vs Corrected GDP): {spearman_costs.correlation:.4f}, p-value = {spearman_costs.pvalue}")

reg_costs: LinregressResult = linregress(df_costs['Costs'], df_costs['Corrected_GDP'])

print(f"Linear regression: slope = {reg_costs.slope:.4f}, intercept = {reg_costs.intercept:.4f}")
print(f"R-squared = {reg_costs.rvalue**2:.4f}, p-value (slope) = {reg_costs.pvalue:.4f}")



Pearson correlation (Costs vs Corrected GDP): -0.3213, p-value = 0.0001
Spearman correlation (Costs vs Corrected GDP): -0.5027, p-value = 3.844757021764013e-10
Linear regression: slope = -1156.2623, intercept = 71729.0313
R-squared = 0.1032, p-value (slope) = 0.0001


In [34]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = df_costs['Costs'], y = df_costs['Corrected_GDP'],
    mode = 'markers',
    marker = dict(size = 8, color = 'orange', opacity = 0.7),
    hovertext = df_costs['Country'],
    hoverinfo = 'text+x+y',
    name = "Страны"
))

fig.update_layout(title = "Стоимость жизни vs Скорректированный ВВП",
                  xaxis_title = "Индекс стоимости жизни",
                  yaxis_title = "Скорректированный ВВП (mio usd)",
                  width = 800, height = 600)

fig.show()


Корреляция Пирсона (-0.321) и Спирмена (-0.503) говорит об отрицательной связи.

p-значения < 0.0001 показывают статистическую значимость зависимости.

Низкий R² = 0.103 указывает, что стоимость жизни объясняет лишь 10.3% вариации скорректированного ВВП.

Но по графику понятно, что все эти результаты связаы с тем, что очень большая часть выборки имеет маленький ВВП и раную цену жизни. Поэтому говорить, что  высокая стоимость жизни приводит к снижению доступного ВВП после корректировки на индекс стоимости жизни

#### Гипотеза 3: Проверка гипотезы: Влияние степени развития институтов на продолжительность жизни

In [35]:
df_institutions: pd.DataFrame = quality_life[['Country', 'Institutions', 'Life expectancy']].dropna()

pearson_inst: PearsonRResult = pearsonr(df_institutions['Institutions'], df_institutions['Life expectancy'])
spearman_inst: SignificanceResult = spearmanr(df_institutions['Institutions'], df_institutions['Life expectancy'])

print(f"Pearson correlation (Inst vs LifeExp): {pearson_inst[0]:.4f}, p-value = {pearson_inst[1]:.4f}")
print(f"Spearman correlation (Inst vs LifeExp): {spearman_inst.correlation:.4f}, p-value = {spearman_inst.pvalue:.4f}")

reg_inst: LinregressResult = linregress(df_institutions['Institutions'], df_institutions['Life expectancy'])

print(f"Linear regression: slope = {reg_inst.slope:.4f}, intercept = {reg_inst.intercept:.4f}")
print(f"R-squared = {reg_inst.rvalue**2:.4f}, p-value (slope) = {reg_inst.pvalue}")

Pearson correlation (Inst vs LifeExp): 0.7806, p-value = 0.0000
Spearman correlation (Inst vs LifeExp): 0.8230, p-value = 0.0000
Linear regression: slope = 0.0000, intercept = 66.3560
R-squared = 0.6094, p-value (slope) = 2.4361054074060197e-29


In [36]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = df_institutions['Institutions'],
    y = df_institutions['Life expectancy'],
    mode = 'markers+text',
    hovertext = df_institutions['Country'],
    hoverinfo = 'text+x+y',
    marker = dict(size = 8, color = 'green', opacity = 0.7),
    name = "Страны"
))
fig.update_layout(
    title = "Институты VS продолжительность жизни",
    xaxis_title = "Степень развития институтов",
    yaxis_title = "Ожидаемое кол-во жизни",
    width = 800,
    height = 600
)
fig.show()

Корреляция Пирсона (0.7806) и Спирмена (0.823) говорит о сильной положительной связи.

Высокий R² = 0.6094 означает, что 61% вариации продолжительности жизни объясняется степенью развития политической и экономической стабильности, правовой системы государства и безопасности.

Вывод: по уровню развития институтов в государстве, можно судить об общем положении дел в экономике, политике и социальной сфере, и соответственно о продолжительности жизни, однако эта связь более слабая по сравнению с качеством медицины, исходя из расчетов.

## **Машинное обучение**

Наконец-то перейдем к самой интересной части проекта. Здесь мы применим наши навыки машинного обучения и попробуем предсказать среднюю продолжительность жизни в различных странах в зависимости от имеющихся параметров.

В качестве переменных параметров логичнее всего будет взять качество развития системы здравоохранения, степень развития институтов и уровень образования в таком порядке приоритетности, исходя из результатов их зависимости с продолжительностью жизни, полученных на этапе визуализации и проверки гипотез. Для начала мы можем построить такую модель и сравнить отклонение полученных значений от реальных данных. Если отклонение будет небольшим, то значит мы оставили наиболее значимые переменные и плодить еще большое количество переменных будет бессмысленно. Иначе, мы можем рассмотреть дополнительно какой-то фактор, который мы забыли учесть на предыдущих этапах.